CLINICAL METADATA PREPROCESSING

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("C:/Users/HP/Downloads/archive/ADNI1_Complete_1Yr_1.5T.csv")
df = df[df['Visit'] == 'sc']
features = ["Image Data ID","Subject","Group","Sex","Age","Visit"]
clinical_data = df[features]
numerical_cols = ['Age']  
for col in numerical_cols:
    clinical_data[col] = clinical_data[col].fillna(clinical_data[col].mean())
clinical_data = clinical_data.dropna(subset=['Sex', 'Group'])
scaler = MinMaxScaler()
clinical_data[numerical_cols] = scaler.fit_transform(clinical_data[numerical_cols])
clinical_data = pd.get_dummies(clinical_data, columns=['Sex'], prefix='Sex')
label_map = {'AD': 0, 'MCI': 1, 'CN': 2}
clinical_data['label'] = clinical_data['Group'].map(label_map)
clinical_data = clinical_data.drop(columns=['Group'])
clinical_data.to_csv("preprocessed_clinical.csv", index=False)
clinical_data.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_16224\3805284833.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_data[col] = clinical_data[col].fillna(clinical_data[col].mean())


,Image Data ID,Subject,Age,Visit,Sex_F,Sex_M,label
0,I118772,018_S_0369,0.600000,sc,False,True,2
1,I119180,067_S_0029,0.257143,sc,False,True,0
3,I119175,053_S_1044,0.314286,sc,False,True,0
4,I118779,018_S_0286,0.314286,sc,False,True,0
7,I81972,131_S_1389,0.600000,sc,False,True,1


PROCESS ALIGNED MRI IMAGES (CONVERT 3D MRI TO 2D PNG)

In [ ]:
import nibabel as nib
import numpy as np
import cv2
import os
from pathlib import Path

mri_dir = "C:/Users/HP/Downloads/archive/Aligned Image"
output_dir = "mri_png"
os.makedirs(output_dir, exist_ok=True)
clinical_data = pd.read_csv("preprocessed_clinical.csv")
image_ids = clinical_data['Image Data ID'].tolist()

#slice_idx=90: Specifies which 2D slice to extract from the 3D MRI volume (default is the 90th slice along the z-axis)
def process_nii_to_png(nii_path, output_path, slice_idx=90, size=(176, 208)):
    img = nib.load(nii_path).get_fdata()
    if slice_idx >= img.shape[2]:
        slice_idx = img.shape[2] // 2 
    #Selects a 2D slice (img[:, :, slice_idx]) from the 3D volume 
    slice_data = img[:, :, slice_idx]
    slice_data = (slice_data - slice_data.min()) / (slice_data.max() - slice_data.min() + 1e-8)
    slice_data = cv2.resize(slice_data, size)
    slice_data = (slice_data * 255).astype(np.uint8)
    cv2.imwrite(output_path, slice_data)

for image_id in image_ids:
    nii_files = list(Path(mri_dir).glob(f"**/{image_id}*.nii*")) 
    if nii_files:
        nii_path = nii_files[0]
        output_path = os.path.join(output_dir, f"{image_id}.png")
        process_nii_to_png(nii_path, output_path)
        print(f"Processed {image_id}")
    else:
        print(f"Missing MRI for {image_id}")

Processed I118772
Processed I119180
Processed I119175
Processed I118779
Processed I81972
Processed I118875
Processed I119137
Processed I119130
Processed I119300
Processed I119123
Processed I138912
Processed I73357
Processed I79948
Processed I119277
Processed I80718
Processed I79838
Processed I118792
Processed I130237
Processed I69113
Processed I79796
Processed I118716
Processed I119264
Processed I119610
Processed I119235
Processed I119087
Processed I118711
Processed I119229
Processed I119510
Processed I119480
Processed I118704
Processed I120057
Processed I119076
Processed I119221
Processed I119067
Processed I119199
Processed I118994
Processed I119737
Processed I119328
Processed I119728
Processed I119192
Processed I118976
Processed I118970
Processed I74591
Processed I118907
Processed I119711
Processed I118686
Processed I118678
Processed I67341
Processed I118671
Processed I67201
Processed I118693
Processed I118687
Processed I121228
Processed I130231
Processed I118680
Processed I140334
Pr

INTEGRATING PREPROCESSED METADATA WITH PROCESSED MRI IMAGES

In [ ]:
import pandas as pd
import os

clinical_data = pd.read_csv("preprocessed_clinical.csv")
mri_dir = "mri_png"
mri_files = [f for f in os.listdir(mri_dir) if f.endswith('.png')]

paired_data = []
for _, row in clinical_data.iterrows():
    image_id = row['Image Data ID']
    mri_path = os.path.join(mri_dir, f"{image_id}.png")
    if os.path.exists(mri_path):
        clin_features = ['Age', 'Sex_M', 'Sex_F']
        clinical = row[clin_features].values.tolist()
        paired_data.append({
            'Image Data ID': image_id,
            'Subject': row['Subject'],
            'mri_path': mri_path,
            'clinical': clinical,
            'label': row['label']
        })
paired_df = pd.DataFrame(paired_data)
paired_df.to_csv("paired_dataset.csv", index=False)
print(f"Paired {len(paired_df)} samples")
print(paired_df.head())

Paired 830 samples
  Image Data ID     Subject             mri_path  \
0       I118772  018_S_0369  mri_png\I118772.png   
1       I119180  067_S_0029  mri_png\I119180.png   
2       I119175  053_S_1044  mri_png\I119175.png   
3       I118779  018_S_0286  mri_png\I118779.png   
4        I81972  131_S_1389   mri_png\I81972.png   

                            clinical  label  
0  [0.5999999999999999, True, False]      2  
1  [0.2571428571428571, True, False]      0  
2  [0.3142857142857143, True, False]      0  
3  [0.3142857142857143, True, False]      0  
4  [0.5999999999999999, True, False]      1  


TRAIN TEST SPLIT AND CHECK BALANCE

In [ ]:
from sklearn.model_selection import train_test_split

paired_df = pd.read_csv("paired_dataset.csv")
train_val, test = train_test_split(paired_df, test_size=0.2, stratify=paired_df['label'], random_state=42)
train, val = train_test_split(train_val, test_size=0.25, stratify=train_val['label'], random_state=42)

train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)
test.to_csv("test.csv", index=False)

print("Train:", train['label'].value_counts(normalize=True))
print("Val:", val['label'].value_counts(normalize=True))
print("Test:", test['label'].value_counts(normalize=True))

Train: label
1    0.485944
2    0.305221
0    0.208835
Name: proportion, dtype: float64
Val: label
1    0.481928
2    0.307229
0    0.210843
Name: proportion, dtype: float64
Test: label
1    0.487952
2    0.307229
0    0.204819
Name: proportion, dtype: float64


ResNet50 CNN + MLP 

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultimodalMRIModel(nn.Module):
    def __init__(self, num_classes=3, clinical_dim=3):
        super(MultimodalMRIModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Reinitialize weights (for better adaptation)
        nn.init.kaiming_normal_(self.resnet.conv1.weight, mode='fan_out', nonlinearity='relu')
        self.resnet.fc = nn.Identity()  # Output: 2048D

        self.clinical_mlp = nn.Sequential(
            nn.Linear(clinical_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        
        # Fusion and classifier
        self.fc = nn.Sequential(
            nn.Linear(2048 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, images, clinical):s
        img_features = self.resnet(images) 
        clin_features = self.clinical_mlp(clinical) 
        combined = torch.cat((img_features, clin_features), dim=1)  
        output = self.fc(combined)  
        return output

DATA LOADING AND AUGMENTATION PIPELINE

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchmetrics import Accuracy, F1Score, AUROC
import pandas as pd
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm
import copy
from pathlib import Path

# custom PyTorch Dataset class to load and preprocess data from CSV files
class ADNIDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        img_path = self.data_df.iloc[idx]['mri_path']
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img / 255.0
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  
        clinical_str = self.data_df.iloc[idx]['clinical']
        clinical = np.array(eval(clinical_str)) 
        clinical = torch.tensor(clinical, dtype=torch.float32)
        label = torch.tensor(self.data_df.iloc[idx]['label'], dtype=torch.long)
        return img, clinical, label

augment = A.Compose([
    A.Rotate(limit=10, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
])

train_dataset = ADNIDataset(pd.read_csv("train.csv"), transform=augment)
val_dataset = ADNIDataset(pd.read_csv("val.csv"), transform=None)
test_dataset = ADNIDataset(pd.read_csv("test.csv"), transform=None)

batch_size = 8
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)


TRAINING PIPELINE

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchmetrics import Accuracy, F1Score, AUROC
from tqdm import tqdm
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalMRIModel(num_classes=3, clinical_dim=3).to(device)

#assigning higher weights to the minority class (AD) in the loss function
paired_df = pd.read_csv("paired_dataset.csv")
class_counts = paired_df['label'].value_counts().sort_index() 
class_weights = torch.tensor(1.0 / class_counts, dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum() * 3
print("Class weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

accuracy = Accuracy(task="multiclass", num_classes=3).to(device)
f1_score = F1Score(task="multiclass", num_classes=3, average="macro").to(device)
auroc = AUROC(task="multiclass", num_classes=3, average="macro").to(device)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    #Uses ReduceLROnPlateau to reduce the learning rate by a factor of 0.1 if validation F1-score doesn’t improve for 5 epochs
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_acc, train_f1, train_auc = 0.0, 0.0, 0.0, 0.0
        train_samples = 0
        print(f"\nEpoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (img, clin, label) in enumerate(tqdm(train_loader, desc="Training Batches")):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            optimizer.zero_grad()
            pred = model(img, clin)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()
            batch_size = img.size(0)
            batch_loss = loss.item()
            batch_acc = accuracy(pred, label).item()
            batch_f1 = f1_score(pred, label).item()
            batch_auc = auroc(pred, label).item()
            train_loss += batch_loss * batch_size
            train_acc += batch_acc * batch_size
            train_f1 += batch_f1 * batch_size
            train_auc += batch_auc * batch_size
            train_samples += batch_size
            
        train_loss /= train_samples
        train_acc /= train_samples
        train_f1 /= train_samples
        train_auc /= train_samples
        
        # Validation
        model.eval()
        val_loss, val_acc, val_f1, val_auc = 0.0, 0.0, 0.0, 0.0
        val_samples = 0
        print(f"\nEpoch {epoch+1}/{num_epochs} - Validation")
        with torch.no_grad():
            for batch_idx, (img, clin, label) in enumerate(tqdm(val_loader, desc="Validation Batches")):
                img, clin, label = img.to(device), clin.to(device), label.to(device)
                pred = model(img, clin)
                loss = criterion(pred, label)
                
                batch_size = img.size(0)
                batch_loss = loss.item()
                batch_acc = accuracy(pred, label).item()
                batch_f1 = f1_score(pred, label).item()
                batch_auc = auroc(pred, label).item()
                
                val_loss += batch_loss * batch_size
                val_acc += batch_acc * batch_size
                val_f1 += batch_f1 * batch_size
                val_auc += batch_auc * batch_size
                val_samples += batch_size
                
              
        
        val_loss /= val_samples
        val_acc /= val_samples
        val_f1 /= val_samples
        val_auc /= val_samples
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}")
        
        # Save epoch model
        epoch_save_path = f"epoch_{epoch+1}_multimodal_mri.pth"
        torch.save(model.state_dict(), epoch_save_path)
        print(f"Saved model: {epoch_save_path}")
        
        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_multimodal_mri.pth")
            print(f"New best model saved with Val F1: {best_f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
        
        scheduler.step(val_f1)
        print(f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        #Stops training if validation F1-score doesn’t improve for patience=10 epochs
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    
    model.load_state_dict(best_model_wts)
    print("Loaded best model weights")
    return model

model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10)

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Class weights: tensor([1.4215, 0.6102, 0.9682])

Epoch 1/50 - Training


Training Batches:   0%|          | 0/63 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Training Batches: 100%|██████████| 63/63 [01:47<00:00,  1.71s/it]



Epoch 1/50 - Validation


Validation Batches: 100%|██████████| 21/21 [00:09<00:00,  2.19it/s]


...


Epoch 49 Summary:
Train - Loss: 0.3186, Acc: 0.8976, F1: 0.8501, AUC: 0.9053
Val   - Loss: 0.9683, Acc: 0.5783, F1: 0.5336, AUC: 0.7456
Saved model: epoch_49_multimodal_mri.pth
Current learning rate: 0.000000

Epoch 50/50 - Training


Training Batches: 100%|██████████| 63/63 [02:35<00:00,  2.48s/it]



Epoch 50/50 - Validation


Validation Batches: 100%|██████████| 21/21 [00:10<00:00,  1.95it/s]


Epoch 50 Summary:
Train - Loss: 0.3351, Acc: 0.8976, F1: 0.8553, AUC: 0.9044
Val   - Loss: 0.9577, Acc: 0.6084, F1: 0.5495, AUC: 0.7460
Saved model: epoch_50_multimodal_mri.pth
Current learning rate: 0.000000
Loaded best model weights


TEST EVALUATION 

In [ ]:
from sklearn.metrics import classification_report

def generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth"):
    model.load_state_dict(torch.load(weights_path))
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = [] 
    with torch.no_grad():
        for img, clin, label in test_loader:
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    
    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("Classification Report:")
    print(report)
    
    return report

generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth")

Classification Report:
              precision    recall  f1-score   support

          AD     0.4750    0.5588    0.5135        34
         MCI     0.7059    0.5926    0.6443        81
          CN     0.5862    0.6667    0.6239        51

    accuracy                         0.6084       166
   macro avg     0.5890    0.6060    0.5939       166
weighted avg     0.6218    0.6084    0.6112       166



'              precision    recall  f1-score   support\n\n          AD     0.4750    0.5588    0.5135        34\n         MCI     0.7059    0.5926    0.6443        81\n          CN     0.5862    0.6667    0.6239        51\n\n    accuracy                         0.6084       166\n   macro avg     0.5890    0.6060    0.5939       166\nweighted avg     0.6218    0.6084    0.6112       166\n'

SMOTE (Synthetic Minority Oversampling Technique) to address the class imbalance

In [ ]:
from imblearn.over_sampling import SMOTE

# Augmentation for synthetic AD MRIs
augment_ad = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_AREA),
    A.Rotate(limit=15, p=0.8),
    A.HorizontalFlip(p=0.8),
    A.RandomBrightnessContrast(p=0.6),
    A.GaussianNoise(p=0.4),
    A.RandomCrop(height=200, width=200, p=0.3)
])

output_dir = "synthetic_mri_ad"
Path(output_dir).mkdir(exist_ok=True)
paired_df = pd.read_csv("paired_dataset.csv")

X_clinical = np.array([eval(row['clinical']) for row in paired_df.itertuples()])
y_labels = paired_df['label'].values

#SMOTE strategy
class_counts = paired_df['label'].value_counts()
max_count = class_counts.max()
sampling_strategy = {
    0: max_count,  
    1: max_count,  
    2: max_count  
}

# Apply SMOTE
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_smote, y_smote = smote.fit_resample(X_clinical, y_labels)

# Generate synthetic MRIs for AD
ad_mri_paths = paired_df[paired_df['label'] == 0]['mri_path'].tolist()
synthetic_mri_paths = []
num_synthetic_ad = max_count - class_counts[0]
for i in range(num_synthetic_ad):
    base_img_path = ad_mri_paths[np.random.randint(len(ad_mri_paths))]
    img = cv2.imread(base_img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = img / 255.0
    augmented = augment_ad(image=img)['image']
    new_path = os.path.join(output_dir, f"synthetic_ad_{i}.png")
    cv2.imwrite(new_path, augmented * 255.0)
    synthetic_mri_paths.append(new_path)

smote_paired_data = []
label_to_rows = paired_df.groupby('label').apply(lambda x: x.to_dict('records')).to_dict()
smote_idx = 0
synthetic_ad_idx = 0

while smote_idx < len(y_smote):
    label = y_smote[smote_idx]
    class_rows = label_to_rows.get(label, [])
    if not class_rows:
        smote_idx += 1
        continue
    row = class_rows[np.random.randint(len(class_rows))]
    
    clinical_data = X_smote[smote_idx]
    if len(clinical_data) >= 3:  
        clinical_data[1] = 1 if clinical_data[1] > 0.5 else 0
        clinical_data[2] = 1 if clinical_data[2] > 0.5 else 0
    clinical_data = clinical_data.tolist()
    
    if smote_idx < len(y_labels):
        image_id = paired_df.iloc[smote_idx]['Image Data ID']
        mri_path = paired_df.iloc[smote_idx]['mri_path']
        label_data = y_labels[smote_idx]
    else:
        # Synthetic sample
        image_id = f"synthetic_{smote_idx}_{row['Image Data ID']}"
        if label == 0 and synthetic_ad_idx < len(synthetic_mri_paths):
            mri_path = synthetic_mri_paths[synthetic_ad_idx]
            synthetic_ad_idx += 1
        else:
            # Reuse MRI for non-AD synthetic samples
            mri_path = row['mri_path']
        label_data = y_smote[smote_idx]
    
    smote_paired_data.append({
        'Image Data ID': image_id,
        'Subject': row['Subject'],
        'mri_path': mri_path,
        'clinical': clinical_data,
        'label': label_data
    })
    smote_idx += 1

paired_df_smote = pd.DataFrame(smote_paired_data)
paired_df_smote.to_csv("paired_dataset_smote.csv", index=False)
print(f"Paired {len(paired_df_smote)} samples after SMOTE")
print(paired_df_smote['label'].value_counts(normalize=True))
print(paired_df_smote.head())

train_val, test = train_test_split(
    paired_df_smote, test_size=0.2, stratify=paired_df_smote['label'], random_state=42
)
train, val = train_test_split(
    train_val, test_size=0.25, stratify=train_val['label'], random_state=42
)

train.to_csv("train_smote.csv", index=False)
val.to_csv("val_smote.csv", index=False)
test.to_csv("test_smote.csv", index=False)

print("Train:", train['label'].value_counts(normalize=True))
print("Val:", val['label'].value_counts(normalize=True))
print("Test:", test['label'].value_counts(normalize=True))

Paired 1209 samples after SMOTE
label
2    0.333333
0    0.333333
1    0.333333
Name: proportion, dtype: float64
  Image Data ID     Subject             mri_path  \
0       I118772  007_S_1222   mri_png\I60003.png   
1       I119180  002_S_0816   mri_png\I40731.png   
2       I119175  123_S_0091  mri_png\I119288.png   
3       I118779  141_S_0852   mri_png\I47744.png   
4        I81972  023_S_0217   mri_png\I31346.png   

                         clinical  label  
0  [0.5999999999999999, 1.0, 0.0]      2  
1  [0.2571428571428571, 1.0, 0.0]      0  
2  [0.3142857142857143, 1.0, 0.0]      0  
3  [0.3142857142857143, 1.0, 0.0]      0  
4  [0.5999999999999999, 1.0, 0.0]      1  
Train: label
2    0.333793
0    0.333793
1    0.332414
Name: proportion, dtype: float64
Val: label
1    0.334711
0    0.334711
2    0.330579
Name: proportion, dtype: float64
Test: label
2    0.334711
1    0.334711
0    0.330579
Name: proportion, dtype: float64


C:\Users\HP\AppData\Local\Temp\ipykernel_27504\1565746625.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_to_rows = paired_df.groupby('label').apply(lambda x: x.to_dict('records')).to_dict()


In [26]:
train_dataset = ADNIDataset(pd.read_csv("train_smote.csv"), transform=augment)
val_dataset = ADNIDataset(pd.read_csv("val_smote.csv"), transform=None)
test_dataset = ADNIDataset(pd.read_csv("test_smote.csv"), transform=None)

TRAINING PIPELINE AFTER SMOTE

In [ ]:
model = MultimodalMRIModel(num_classes=3, clinical_dim=3).to(device)

# Class weights (recomputed for balanced dataset)
paired_df = pd.read_csv("paired_dataset_smote.csv")
class_counts = paired_df['label'].value_counts().sort_index()  
class_weights = torch.tensor(1.0 / class_counts, dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum() * 3
print("Class weights:", class_weights)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_acc, train_f1, train_auc = 0.0, 0.0, 0.0, 0.0
        train_samples = 0
        print(f"\nEpoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (img, clin, label) in enumerate(tqdm(train_loader, desc="Training Batches")):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            optimizer.zero_grad()
            pred = model(img, clin)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()
            
            batch_size = img.size(0)
            batch_loss = loss.item()
            batch_acc = accuracy(pred, label).item()
            batch_f1 = f1_score(pred, label).item()
            batch_auc = auroc(pred, label).item()
            
            train_loss += batch_loss * batch_size
            train_acc += batch_acc * batch_size
            train_f1 += batch_f1 * batch_size
            train_auc += batch_auc * batch_size
            train_samples += batch_size
        
        train_loss /= train_samples
        train_acc /= train_samples
        train_f1 /= train_samples
        train_auc /= train_samples
        
        # Validation
        model.eval()
        val_loss, val_acc, val_f1, val_auc = 0.0, 0.0, 0.0, 0.0
        val_samples = 0
        
        print(f"\nEpoch {epoch+1}/{num_epochs} - Validation")
        with torch.no_grad():
            for batch_idx, (img, clin, label) in enumerate(tqdm(val_loader, desc="Validation Batches")):
                img, clin, label = img.to(device), clin.to(device), label.to(device)
                pred = model(img, clin)
                loss = criterion(pred, label)
                
                batch_size = img.size(0)
                batch_loss = loss.item()
                batch_acc = accuracy(pred, label).item()
                batch_f1 = f1_score(pred, label).item()
                batch_auc = auroc(pred, label).item()
                
                val_loss += batch_loss * batch_size
                val_acc += batch_acc * batch_size
                val_f1 += batch_f1 * batch_size
                val_auc += batch_auc * batch_size
                val_samples += batch_size
        
        val_loss /= val_samples
        val_acc /= val_samples
        val_f1 /= val_samples
        val_auc /= val_samples
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}")
        
        epoch_save_path = f"epoch_{epoch+1}_multimodal_mri.pth"
        torch.save(model.state_dict(), epoch_save_path)
        print(f"Saved model: {epoch_save_path}")
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_multimodal_mri.pth")
            print(f"New best model saved with Val F1: {best_f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
        
        scheduler.step(val_f1)
        print(f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    
    model.load_state_dict(best_model_wts)
    print("Loaded best model weights")
    return model

def generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth"):
    model.load_state_dict(torch.load(weights_path))
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for img, clin, label in test_loader:
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    
    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("Classification Report:")
    print(report)
    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_preds
    })
    results_df.to_csv("test_predictions_resnet.csv", index=False)
    print("Saved predictions to test_predictions_resnet.csv")
    return report, results_df

model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10)
report, results_df = generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth")

Using device: cpu


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Class weights: tensor([1., 1., 1.])

Epoch 1/50 - Training


Training Batches:   4%|▍         | 4/91 [00:04<01:41,  1.17s/it]C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Training Batches:  79%|███████▉  | 72/91 [01:33<00:26,  1.39s/it]C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
Training Batches: 100%|██████████| 91/91 [01:58<00:00,  1.30s/it]



Epoch 1/50 - Validation


...


Epoch 49 Summary:
Train - Loss: 0.0493, Acc: 0.9876, F1: 0.9782, AUC: 0.9595
Val   - Loss: 0.6288, Acc: 0.8306, F1: 0.8027, AUC: 0.8793
Saved model: epoch_49_multimodal_mri.pth
Current learning rate: 0.000000

Epoch 50/50 - Training


Training Batches: 100%|██████████| 91/91 [05:23<00:00,  3.55s/it]



Epoch 50/50 - Validation


Validation Batches: 100%|██████████| 31/31 [00:32<00:00,  1.05s/it]



Epoch 50 Summary:
Train - Loss: 0.0571, Acc: 0.9807, F1: 0.9686, AUC: 0.9627
Val   - Loss: 0.6276, Acc: 0.8471, F1: 0.8204, AUC: 0.8772
Saved model: epoch_50_multimodal_mri.pth
Current learning rate: 0.000000
Loaded best model weights
Classification Report:
              precision    recall  f1-score   support

          AD     0.9000    0.9000    0.9000        80
         MCI     0.8696    0.7407    0.8000        81
          CN     0.7957    0.9136    0.8506        81

    accuracy                         0.8512       242
   macro avg     0.8551    0.8514    0.8502       242
weighted avg     0.8549    0.8512    0.8500       242

Saved predictions to test_predictions_resnet.csv


DenseNet121 MODEL

In [ ]:
from torchvision.models import densenet121, DenseNet121_Weights

class MultimodalDenseNet(nn.Module):
    def __init__(self, num_classes=3, clinical_dim=3):
        super(MultimodalDenseNet, self).__init__()
        self.densenet = densenet121(weights=DenseNet121_Weights.DEFAULT)
        self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        nn.init.kaiming_normal_(self.densenet.features.conv0.weight, mode='fan_out', nonlinearity='relu')
        self.densenet.classifier = nn.Identity()  # Output: 1024D
        
        self.clinical_mlp = nn.Sequential(
            nn.Linear(clinical_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        # Fusion
        self.fc = nn.Sequential(
            nn.Linear(1024 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, images, clinical):
        img_features = self.densenet(images)  
        clin_features = self.clinical_mlp(clinical) 
        combined = torch.cat((img_features, clin_features), dim=1)
        output = self.fc(combined)
        return output

TRAINING PIPELINE

In [ ]:
model = MultimodalDenseNet(num_classes=3, clinical_dim=3).to(device)

paired_df = pd.read_csv("paired_dataset_smote.csv")
class_counts = paired_df['label'].value_counts().sort_index()
class_weights = torch.tensor(1.0 / class_counts, dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum() * 3
print("Class weights:", class_weights)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_acc, train_f1, train_auc = 0.0, 0.0, 0.0, 0.0
        train_samples = 0
        
        print(f"\nEpoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (img, clin, label) in enumerate(tqdm(train_loader, desc="Training Batches")):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            optimizer.zero_grad()
            pred = model(img, clin)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()
            
            batch_size = img.size(0)
            batch_loss = loss.item()
            batch_acc = accuracy(pred, label).item()
            batch_f1 = f1_score(pred, label).item()
            batch_auc = auroc(pred, label).item()
            
            train_loss += batch_loss * batch_size
            train_acc += batch_acc * batch_size
            train_f1 += batch_f1 * batch_size
            train_auc += batch_auc * batch_size
            train_samples += batch_size
        
        train_loss /= train_samples
        train_acc /= train_samples
        train_f1 /= train_samples
        train_auc /= train_samples
        
        model.eval()
        val_loss, val_acc, val_f1, val_auc = 0.0, 0.0, 0.0, 0.0
        val_samples = 0
        
        print(f"\nEpoch {epoch+1}/{num_epochs} - Validation")
        with torch.no_grad():
            for batch_idx, (img, clin, label) in enumerate(tqdm(val_loader, desc="Validation Batches")):
                img, clin, label = img.to(device), clin.to(device), label.to(device)
                pred = model(img, clin)
                loss = criterion(pred, label)
                
                batch_size = img.size(0)
                batch_loss = loss.item()
                batch_acc = accuracy(pred, label).item()
                batch_f1 = f1_score(pred, label).item()
                batch_auc = auroc(pred, label).item()
                
                val_loss += batch_loss * batch_size
                val_acc += batch_acc * batch_size
                val_f1 += batch_f1 * batch_size
                val_auc += batch_auc * batch_size
                val_samples += batch_size
        
        val_loss /= val_samples
        val_acc /= val_samples
        val_f1 /= val_samples
        val_auc /= val_samples
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}")
        
        epoch_save_path = f"epoch_{epoch+1}_multimodal_densenet.pth"
        torch.save(model.state_dict(), epoch_save_path)
        print(f"Saved model: {epoch_save_path}")
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_multimodal_densenet.pth")
            print(f"New best model saved with Val F1: {best_f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
        
        scheduler.step(val_f1)
        print(f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    
    model.load_state_dict(best_model_wts)
    print("Loaded best model weights")
    return model

def generate_classification_report(model, test_loader, device, weights_path="best_multimodal_densenet.pth"):
    model.load_state_dict(torch.load(weights_path))
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for img, clin, label in test_loader:
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    
    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("DenseNet Classification Report:")
    print(report)
    
    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_preds
    })
    results_df.to_csv("test_predictions_densenet.csv", index=False)
    print("Saved predictions to test_predictions_densenet.csv")
    
    return report, results_df

model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10)
report, results_df = generate_classification_report(model, test_loader, device)

Using device: cpu


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:21<00:00, 1.52MB/s]


Class weights: tensor([1., 1., 1.])

Epoch 1/50 - Training


Training Batches:  12%|█▏        | 11/91 [00:16<01:57,  1.47s/it]C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Training Batches:  99%|█████████▉| 90/91 [02:07<00:01,  1.38s/it]C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
Training Batches: 100%|██████████| 91/91 [02:09<00:00,  1.42s/it]



Epoch 1/50 - Validation


...


Epoch 49 Summary:
Train - Loss: 0.0443, Acc: 0.9903, F1: 0.9800, AUC: 0.9559
Val   - Loss: 0.7355, Acc: 0.8017, F1: 0.7721, AUC: 0.8810
Saved model: epoch_49_multimodal_densenet.pth
Current learning rate: 0.000001

Epoch 50/50 - Training


Training Batches: 100%|██████████| 91/91 [04:14<00:00,  2.80s/it]



Epoch 50/50 - Validation


Validation Batches: 100%|██████████| 31/31 [00:18<00:00,  1.66it/s]



Epoch 50 Summary:
Train - Loss: 0.0645, Acc: 0.9848, F1: 0.9725, AUC: 0.9620
Val   - Loss: 0.7407, Acc: 0.8017, F1: 0.7721, AUC: 0.8799
Saved model: epoch_50_multimodal_densenet.pth
Current learning rate: 0.000001
Loaded best model weights
DenseNet Classification Report:
              precision    recall  f1-score   support

          AD     0.8256    0.8875    0.8554        80
         MCI     0.8060    0.6667    0.7297        81
          CN     0.7640    0.8395    0.8000        81

    accuracy                         0.7975       242
   macro avg     0.7985    0.7979    0.7951       242
weighted avg     0.7984    0.7975    0.7948       242

Saved predictions to test_predictions_densenet.csv


CLASSIFICATION REPORTS

In [ ]:
def generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth"):
    model.load_state_dict(torch.load(weights_path))
    print(f"Loaded model weights from {weights_path}")
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []
    print("Running inference on test set...")
    with torch.no_grad():
        for img, clin, label in tqdm(test_loader, desc="Test Batches"):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("\nClassification Report:")
    print(report)
    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_preds
    })
    results_df.to_csv("test_predictions_resnet.csv", index=False)
    return report, results_df

model = MultimodalMRIModel(num_classes=3, clinical_dim=3)
report, results_df = generate_classification_report(model, test_loader, device, weights_path="best_multimodal_mri.pth")

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded model weights from best_multimodal_mri.pth
Running inference on test set...


Test Batches: 100%|██████████| 31/31 [00:31<00:00,  1.00s/it]


Classification Report:
              precision    recall  f1-score   support

          AD     0.9000    0.9000    0.9000        80
         MCI     0.8696    0.7407    0.8000        81
          CN     0.7957    0.9136    0.8506        81

    accuracy                         0.8512       242
   macro avg     0.8551    0.8514    0.8502       242
weighted avg     0.8549    0.8512    0.8500       242

Saved classification report to classification_report.txt
Saved predictions to test_predictions_resnet.csv


In [ ]:
def generate_classification_report(model, test_loader, device, weights_path="best_multimodal_densenet.pth"):
    model.load_state_dict(torch.load(weights_path))
    print(f"Loaded model weights from {weights_path}")
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []
    print("Running inference on test set...")
    with torch.no_grad():
        for img, clin, label in tqdm(test_loader, desc="Test Batches"):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("\nDenseNet Classification Report:")
    print(report)
    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_preds
    })
    results_df.to_csv("test_predictions_densenet.csv", index=False)
    print("Saved predictions to test_predictions_densenet.csv")
    return report, results_df

model = MultimodalDenseNet(num_classes=3, clinical_dim=3)
report, results_df = generate_classification_report(model, test_loader, device, weights_path="best_multimodal_densenet.pth")

Using device: cpu
Loaded model weights from best_multimodal_densenet.pth
Running inference on test set...


Test Batches: 100%|██████████| 31/31 [00:12<00:00,  2.51it/s]


DenseNet Classification Report:
              precision    recall  f1-score   support

          AD     0.8256    0.8875    0.8554        80
         MCI     0.8060    0.6667    0.7297        81
          CN     0.7640    0.8395    0.8000        81

    accuracy                         0.7975       242
   macro avg     0.7985    0.7979    0.7951       242
weighted avg     0.7984    0.7975    0.7948       242

Saved classification report to classification_report_densenet.txt
Saved predictions to test_predictions_densenet.csv


In [ ]:
from scipy.stats import mode

class MultimodalEfficientNet(nn.Module):
    def __init__(self, num_classes=3, clinical_dim=3):
        super(MultimodalEfficientNet, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, in_chans=1)
        for param in self.efficientnet.conv_stem.parameters():
            param.requires_grad = False
        for param in self.efficientnet.bn1.parameters():
            param.requires_grad = False
        self.efficientnet.classifier = nn.Identity()
        self.feature_dim = 1280
        self.clinical_mlp = nn.Sequential(
            nn.Linear(clinical_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(1280 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, clinical):
        img_features = self.efficientnet(images)
        clin_features = self.clinical_mlp(clinical)
        combined = torch.cat((img_features, clin_features), dim=1)
        output = self.fc(combined)
        return output


ENSEMBLE CLASSIFICATION

In [ ]:
test_df = pd.read_csv("test_smote.csv")
print(f"Test samples: {len(test_df)}")
test_dataset = ADNIDataset(test_df, transform=test_transform)

batch_size = 8
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,  
    pin_memory=False
)

def generate_ensemble_classification_report(models, weights_paths, test_loader, device):
    resnet_model = models[0]
    densenet_model = models[1]
    efficientnet_model = models[2]
    resnet_model.load_state_dict(torch.load(weights_paths[0], map_location=torch.device('cpu')))
    densenet_model.load_state_dict(torch.load(weights_paths[1], map_location=torch.device('cpu')))
    efficientnet_model.load_state_dict(torch.load(weights_paths[2], map_location=torch.device('cpu')))
    print(f"Loaded weights: {weights_paths}")
    resnet_model.to(device)
    densenet_model.to(device)
    efficientnet_model.to(device)
    
    # Set to evaluation mode
    resnet_model.eval()
    densenet_model.eval()
    efficientnet_model.eval()
    all_preds_soft = []
    all_labels = []
    all_probs = []  
    print("Running ensemble inference on test set...")
    with torch.no_grad():
        for img, clin, label in tqdm(test_loader, desc="Test Batches"):
            img, clin, label = img.to(device), clin.to(device), label.to(device) 
            # Get predictions from each model
            pred_resnet = resnet_model(img, clin)
            pred_densenet = densenet_model(img, clin)
            pred_efficientnet = efficientnet_model(img, clin)
            # Converts logits to probabilities via softmax
            probs_resnet = torch.softmax(pred_resnet, dim=1)
            probs_densenet = torch.softmax(pred_densenet, dim=1)
            probs_efficientnet = torch.softmax(pred_efficientnet, dim=1)
            #Soft voting: average probabilities
            avg_probs = (probs_resnet + probs_densenet + probs_efficientnet) / 3
            pred_soft = torch.argmax(avg_probs, dim=1)
            all_preds_soft.extend(pred_soft.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
            all_probs.extend(avg_probs.cpu().numpy())
    
    report_soft = classification_report(all_labels, all_preds_soft, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("\nEnsemble Classification Report (Soft Voting):")
    print(report_soft)
    output_report_path = "classification_report_ensemble.txt"
    with open(output_report_path, "w") as f:
        f.write("Ensemble Classification Report (Soft Voting):\n")
        f.write(report_soft) 
    print(f"Saved classification report to {output_report_path}")
    output_csv_path = "test_predictions_ensemble.csv"
    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label_soft': all_preds_soft,
    })
    results_df.to_csv(output_csv_path, index=False)
    print(f"Saved predictions to {output_csv_path}")
    
    return report_soft,  results_df

resnet_model = MultimodalMRIModel(num_classes=3, clinical_dim=3)
densenet_model = MultimodalDenseNet(num_classes=3, clinical_dim=3)
efficientnet_model = MultimodalEfficientNet(num_classes=3, clinical_dim=3)
models = [resnet_model, densenet_model, efficientnet_model]

# Model weights paths
weights_paths = [
    "best_multimodal_mri.pth",
    "best_multimodal_densenet.pth",
    "best_multimodal_efficientnet.pth"
]
report_soft,  results_df = generate_ensemble_classification_report(models, weights_paths, test_loader, device)

Using device: cpu
Test samples: 242
Loaded weights: ['best_multimodal_mri.pth', 'best_multimodal_densenet.pth', 'best_multimodal_efficientnet.pth']
Running ensemble inference on test set...


Test Batches: 100%|██████████| 31/31 [01:03<00:00,  2.06s/it]


Ensemble Classification Report (Soft Voting):
              precision    recall  f1-score   support

          AD     0.8452    0.8875    0.8659        80
         MCI     0.8261    0.7037    0.7600        81
          CN     0.7416    0.8148    0.7765        81

    accuracy                         0.8017       242
   macro avg     0.8043    0.8020    0.8008       242
weighted avg     0.8041    0.8017    0.8005       242

Saved classification report to classification_report_ensemble.txt
Saved predictions to test_predictions_ensemble.csv
